In [1]:
import time
import math
import scipy.sparse as sp
import scipy.linalg as la
from scipy.sparse import csc_matrix, linalg as sla

import numpy as np



## LU decomposition of sparse matrix 
LU decomposition of sparse matrix is not sparse in general. 

In [2]:
N = 1000  # N, number of nonzero elements in a=O(N)
d = 0.005
a = sp.rand(N, N, d)+sp.identity(N)

lu = sla.splu(a)
print('number of non-zeros in L&U:', lu.nnz)
print('number of non-zeros in a:', a.nnz)


number of non-zeros in L&U: 395751
number of non-zeros in a: 5998


/home/cesar/anaconda3/lib/python3.8/site-packages/scipy/sparse/linalg/dsolve/linsolve.py:318: SparseEfficiencyWarning: splu requires CSC matrix format
  warn('splu requires CSC matrix format', SparseEfficiencyWarning)


## Preconditioning Conjugate Gradient 
The matrix $A$ is a $n\times n$ symmetric positive definite matrix with all zeros except 
$a_{ii} = 0.5 + \sqrt i$ on the diagonal,
$a_{ij} = 1$ 
on the sub- and superdiagonal, and 
$a_{ij} = 1$ on the 100th sub- and superdiagonals, i.e., for $|i − j| = 100$. 
The right-hand side vector is $b = [1, \cdots, 1]^T$. 
We observe that the basic CG algorithm converges very slowly, whereas the Jacobi-preconditioned method converges much faster.

In [3]:
n = 1000000
diagonal = [1/2+math.sqrt(i) for i in range(n)]
subdiag = np.ones(n-1)
subdiag100 = np.ones(n-100)
diagonals = [diagonal, subdiag, subdiag, subdiag100, subdiag100]

In [4]:
A = sp.diags(diagonals, [0,1,-1, 100, -100]).tocsc()
b = np.ones(n)

In [5]:
t = time.time()
lu = sla.splu(A)
x = lu.solve(b)
elapsed = time.time() - t
print('Computational time of solving by LU decomposition:', elapsed)

Computational time of solving by LU decomposition: 3.5468759536743164


In [6]:
t = time.time()
y,k = sla.cg(A, b)
elapsed = time.time() - t
print('Computational time of solving by Conjugate Gradient:', elapsed)

Computational time of solving by Conjugate Gradient: 4.7844014167785645


In [7]:
t = time.time()
M = sp.diags([1/d for d in diagonal]).tocsc()
y,k = sla.cg(A, b, M = M)
elapsed = time.time() - t
print('Computational time of solving by Conjugate Gradient with Jacobi preconditioning:', elapsed)

Computational time of solving by Conjugate Gradient with Jacobi preconditioning: 0.41562318801879883
